In [21]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [22]:
from nltk import ngrams
from collections import Counter, defaultdict
import numpy as np

# Preprocesamiento

In [23]:
import re

def preprocess_corpus(corpus: list[str]) -> list[str]:
    """Función de preprocesamiento

    Agrega tokens de inicio y fin, normaliza todo a minusculas
    """
    preprocessed_corpus = []
    for sent in corpus:
        result = [word.lower() for word in sent]
        # Al final de la oración
        result.append("<EOS>")
        result.insert(0, "<BOS>")
        preprocessed_corpus.append(result)
    return preprocessed_corpus

In [24]:
def get_words_freqs(corpus: list[list[str]]):
    words_freqs = {}
    for sentence in corpus:
        for word in sentence:
            words_freqs[word] = words_freqs.get(word, 0) + 1
    return words_freqs

In [25]:
UNK_LABEL = "<UNK>"
def get_words_indexes(words_freqs: dict) -> dict:
    result = {}
    for idx, word in enumerate(words_freqs.keys()):
        # Happax legomena happends
        if words_freqs[word] == 1:
            # Temp index for unknowns
            result[UNK_LABEL] = len(words_freqs)
        else:
            result[word] = idx

    return {word: idx for idx, word in enumerate(result.keys())}, {idx: word for idx, word in enumerate(result.keys())}

In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
%cd drive/MyDrive/Practica8

[Errno 2] No such file or directory: 'drive/MyDrive/Practica8'
/content/drive/MyDrive/Practica8


In [30]:
def lee_corpus(file_name: str) -> str:
    with open(file_name, "r") as f:
        return f.read()

quijote = lee_corpus("El_Quijote.txt")
sents = nltk.sent_tokenize(quijote)
corpus = [nltk.word_tokenize(sent) for sent in sents]

In [31]:
corpus = preprocess_corpus(corpus)

In [158]:
print(len(corpus))

9513


In [32]:
words_freqs = get_words_freqs(corpus)

In [33]:
words_indexes, index_to_word = get_words_indexes(words_freqs)

In [34]:
def get_word_id(words_indexes: dict, word: str) -> int:
    unk_word_id = words_indexes[UNK_LABEL]
    return words_indexes.get(word, unk_word_id)

In [35]:
def get_train_test_data(corpus: list[list[str]], words_indexes: dict, n: int) -> tuple[list, list]:
    x_train = []
    y_train = []
    for sent in corpus:
        n_grams = ngrams(sent, n)
        for w1, w2, w3 in n_grams:
            x_train.append([get_word_id(words_indexes, w1), get_word_id(words_indexes, w2)])
            y_train.append([get_word_id(words_indexes, w3)])
    return x_train, y_train

In [36]:
# cargamos bibliotecas
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import time

In [37]:
# Setup de parametros
EMBEDDING_DIM = 200
CONTEXT_SIZE = 2
BATCH_SIZE = 256
H = 100
torch.manual_seed(19)
# Tamaño del Vocabulario
V = len(words_indexes)

In [38]:
x_train, y_train = get_train_test_data(corpus, words_indexes, n=3)

In [39]:
train_set = np.concatenate((x_train, y_train), axis=1)
# partimos los datos de entrada en batches
train_loader = DataLoader(train_set, batch_size = BATCH_SIZE)

In [40]:
# Trigram Neural Network Model
class TrigramModel(nn.Module):
    """Clase padre: https://pytorch.org/docs/stable/generated/torch.nn.Module.html"""

    def __init__(self, vocab_size, embedding_dim, context_size, h):
        super(TrigramModel, self).__init__()
        self.context_size = context_size
        self.embedding_dim = embedding_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, h)
        self.linear2 = nn.Linear(h, vocab_size)

    def forward(self, inputs):
        # x': concatenation of x1 and x2 embeddings   -->
        #self.embeddings regresa un vector por cada uno de los índices que se les pase como entrada. view() les cambia el tamaño para concatenarlos
        embeds = self.embeddings(inputs).view((-1,self.context_size * self.embedding_dim))
        # h: tanh(W_1.x' + b)  -->
        out = torch.tanh(self.linear1(embeds))
        # W_2.h                 -->
        out = self.linear2(out)
        # log_softmax(W_2.h)      -->
        # dim=1 para que opere sobre renglones, pues al usar batchs tenemos varios vectores de salida
        log_probs = F.log_softmax(out, dim=1)

        return log_probs

# Entrenamiento

In [20]:
# 1. Pérdida. Negative log-likelihood loss
loss_function = nn.NLLLoss()

#Otras opciones de función de pérdida (tendrían que usar softmax sin log):
#nn.CrossEntropyLoss()


# 2. Instanciar el modelo
model = TrigramModel(V, EMBEDDING_DIM, CONTEXT_SIZE, H)

# 3. Optimización. ADAM optimizer
optimizer = optim.Adam(model.parameters(), lr = 2e-3)

#Otras opciones de optimizador:
#optimizer = optim.SGD(model.parameters(), lr=0.1)


# ------------------------- TRAIN & SAVE MODEL ------------------------
# En la práctica sólo correremos una epoch por restricciones de recursos
EPOCHS = 1
for epoch in range(EPOCHS):
    st = time.time()
    print("\n--- Training model Epoch: {} ---".format(epoch))
    for it, data_tensor in enumerate(train_loader):
        context_tensor = data_tensor[:,0:2]
        target_tensor = data_tensor[:,2]

        model.zero_grad() #reinicializar los gradientes
        #FORWARD:
        # get log probabilities over next words
        log_probs = model(context_tensor)


        # compute loss function
        loss = loss_function(log_probs, target_tensor)

        #BACKWARD:
        # backward pass and update gradient
        loss.backward()
        optimizer.step()

        if it % 500 == 0:
            print("Training Iteration {} of epoch {} complete. Loss: {}; Time taken (s): {}".format(it, epoch, loss.item(), (time.time()-st)))
            st = time.time()
            #barch_size x len(vocab)

    # saving model
    model_path = 'model_{}.dat'.format(epoch)
    torch.save(model.state_dict(), model_path)
    print(f"Model saved for epoch={epoch} at {model_path}")


--- Training model Epoch: 0 ---
Training Iteration 0 of epoch 0 complete. Loss: 9.429872512817383; Time taken (s): 0.2951540946960449
Training Iteration 500 of epoch 0 complete. Loss: 5.698538780212402; Time taken (s): 44.24118232727051
Training Iteration 1000 of epoch 0 complete. Loss: 5.12728214263916; Time taken (s): 42.260573625564575
Training Iteration 1500 of epoch 0 complete. Loss: 4.884504318237305; Time taken (s): 41.96507930755615
Model saved for epoch=0 at model_0.dat


In [41]:
def get_model(path: str) -> TrigramModel:
    model_loaded = TrigramModel(V, EMBEDDING_DIM, CONTEXT_SIZE, H)
    model_loaded.load_state_dict(torch.load(path))
    model_loaded.eval()
    return model_loaded

In [42]:
PATH = "model_0.dat"

In [43]:
model = get_model(PATH)
W1 = "<BOS>"
W2 = "mi"

IDX1 = get_word_id(words_indexes, W1)
IDX2 = get_word_id(words_indexes, W2)

#Obtenemos Log probabidades p(W3|W2,W1)
probs = model(torch.tensor([[IDX1,  IDX2]])).detach().tolist()

In [44]:
# Creamos diccionario con {idx: logprob}
model_probs = {}
for idx, p in enumerate(probs[0]):
  model_probs[idx] = p

# Sort:
model_probs_sorted = sorted(((prob, idx) for idx, prob in model_probs.items()), reverse=True)

# Printing word  and prob (retrieving the idx):
topcandidates = 0
for prob, idx in model_probs_sorted:
  #Retrieve the word associated with that idx
  word = index_to_word[idx]
  print(idx, word, prob)

  topcandidates += 1

  if topcandidates > 100:
    break

135 señor -2.291330337524414
228 vida -2.7545151710510254
300 señora -2.9274682998657227
142 amo -3.1409263610839844
857 padre -3.5086264610290527
291 mujer -3.5320324897766113
782 parecer -3.5429940223693848
125 escudero -3.6744327545166016
481 hija -4.088214874267578
63 buen -4.133939743041992
889 alma -4.155529499053955
469 nombre -4.305261611938477
1462 patria -4.5872039794921875
336 casa -4.608495712280273
188 intención -4.747317314147949
548 consejo -4.759873390197754
587 <UNK> -4.768007278442383
2063 marido -4.772164344787598
1354 ventura -4.835598945617676
178 historia -5.045218467712402
275 sobrina -5.1450018882751465
2837 voluntad -5.203191757202148
806 hijo -5.235255718231201
143 cuerpo -5.320702075958252
276 ama -5.438802719116211
23 que -5.439906120300293
315 muerte -5.449024200439453
797 deseo -5.473878860473633
1204 tal -5.487883567810059
475 camino -5.503872871398926
430 bien -5.539588928222656
602 madre -5.557007789611816
838 lugar -5.567862033843994
886 amigo -5.62757

# Generación de lenguaje

In [62]:
def get_likely_words(model: TrigramModel, context: str, words_indexes: dict, index_to_word: dict, top_count: int=10) -> list[tuple]:
    model_probs = {}
    words = context.split()
    idx_word_1 = get_word_id(words_indexes, words[0])
    idx_word_2 = get_word_id(words_indexes, words[1])
    probs = model(torch.tensor([[idx_word_1, idx_word_2]])).detach().tolist()

    for idx, p in enumerate(probs[0]):
        model_probs[idx] = p

    # Strategy: Sort and get top-K words to generate text
    return sorted(((prob, index_to_word[idx]) for idx, prob in model_probs.items()), reverse=True)[:top_count]

In [64]:
print(get_likely_words(model, "el día", words_indexes, index_to_word))

[(-1.0571527481079102, ','), (-1.8467793464660645, 'de'), (-2.6491775512695312, 'que'), (-2.942416191101074, 'y'), (-3.1142191886901855, '.'), (-3.360020160675049, 'del'), (-3.906507968902588, ';'), (-3.9203553199768066, 'a'), (-4.243419170379639, 'en'), (-4.46452522277832, 'se')]


In [115]:
from random import randint

def get_next_word_random(words: list[tuple[float, str]]) -> str:
    # From a top-K list of words get a random word
    return words[randint(0, len(words)-1)][1]

In [116]:
from random import randint

def get_next_word(words: list[tuple[float, str]]) -> str:
    # Estrategia propuesta
    if (randint(0,5) < 3):
      return words[randint(0, 2)][1]
    elif (randint(0,5) < 5):
      return words[randint(3, 5)][1]
    else:
      return words[randint(6, 9)][1]



In [129]:
MAX_TOKENS = 30
TOP_COUNT = 10
def generate_text_random(model: TrigramModel, history: str, words_indexes: dict, index_to_word: dict, tokens_count: int=0) -> None:
    next_word = get_next_word_random(get_likely_words(model, history, words_indexes, index_to_word, top_count=TOP_COUNT))
    print(next_word, end=" ")
    tokens_count += 1
    if tokens_count == MAX_TOKENS or next_word == "<EOS>":
        return
    generate_text_random(model, history.split()[1]+ " " + next_word, words_indexes, index_to_word, tokens_count)

In [118]:
MAX_TOKENS = 30
TOP_COUNT = 10
def generate_text(model: TrigramModel, history: str, words_indexes: dict, index_to_word: dict, tokens_count: int=0) -> None:
    next_word = get_next_word(get_likely_words(model, history, words_indexes, index_to_word, top_count=TOP_COUNT))
    print(next_word, end=" ")
    tokens_count += 1
    if tokens_count == MAX_TOKENS or next_word == "<EOS>":
        return
    generate_text(model, history.split()[1]+ " " + next_word, words_indexes, index_to_word, tokens_count)

# Ejemplos

Con la estrategia original de la práctica

In [132]:
# Ejemplo 1
sentence = "<BOS> Entonces"
print(sentence, end=" ")
generate_text_random(model, sentence, words_indexes, index_to_word)

<BOS> Entonces , que me <UNK> , el pastor de ver si el señor bachiller . el señor : el uno <UNK> y a otros a las de los azotes en la 

In [135]:
# Ejemplo 2
sentence = "Había una"
print(sentence, end=" ")
generate_text_random(model, sentence, words_indexes, index_to_word)

Había una mosca de su aldea . de las armas ; porque se <UNK> el bachiller que los tres . , señor gentilhombre a un año ; porque se le ha sucedido 

In [154]:
# Ejemplo 3
sentence = "el señor"
print(sentence, end=" ")
generate_text_random(model, sentence, words_indexes, index_to_word)

el señor : el cura la vida : `` no es mi alma que , y , <UNK> de un caballero <UNK> ; y que , <UNK> el cielo don juan a 

Con la estrategia propuesta

In [141]:
# Ejemplo 1
sentence = "<BOS> Entonces"
print(sentence, end=" ")
generate_text(model, sentence, words_indexes, index_to_word)

<BOS> Entonces la mancha de don gregorio de su alma ; porque yo no lo consintió el mundo ; pero que no soy de la noche del mundo , porque se ha 

In [145]:
# Ejemplo 2
sentence = "Había una"
print(sentence, end=" ")
generate_text(model, sentence, words_indexes, index_to_word)

Había una vez , como no soy la vida , que ya estaba el mundo ; porque en las cuales del mundo . de mi señora : y , por cierto , 

In [155]:
# Ejemplo 3
sentence = "el señor"
print(sentence, end=" ")
generate_text(model, sentence, words_indexes, index_to_word)

el señor , como lo es —respondió el mozo— ; mas a don gaiferos que no es bien el cura que el señor caballero del pueblo : — no me ha hecho 

# Información de setup de entrenamiento


# Setup de parametros
Dimensión de embeddings: 200

Tamaño del contexto: 2

Tamaño de la capa oculta: 100

Tamaño del Batch: 256

Cantidad de oraciones para entrenamiento: 9513

# Estrategia propuesta
En lugar de elegir completamente aleatorio entre las 10 opciones más probables, se elige un número aleatorio entre 0 y 5. Si es menor que 3, se elige una opción aleatoria entre las 3 más probables, si está entre 3 y 4, se elige entre las siguientes 3 más probables, y si es 5 se elegi entre las otras 4.

Con esto espero que cualquiera de las opciones puedan salir, pero sea más probable que salgan las que tienen mayor probabilidad según el modelo.

Al comparar los resultados de la estrategia anterior con los de esta estrategia, no parece haber una gran diferencia; los resultados me siguen pareciendo muy aleatorios, probablemente porque fueron muy pocos datos de entrenamiento, así que las probabilidades del modelo no son muy precisas.